In [1]:
import duckdb
import altair as alt
import polars as pl
import yfinance as yf

c_datetime_local = "datetime_local"
c_close = "close"
c_symbol = "symbol"
c_resolution = "resolution"
EXPORT_PATH = "../../assets/images/benner"

In [36]:
resolution = "1wk"

df_ohlc = pl.from_dataframe(yf.Ticker("^GSPC").history(period="max", interval=resolution).reset_index()).with_columns(
    pl.col("Date").alias(c_datetime_local),
    pl.col("Close").alias(c_close),
    pl.lit("^GSPC").alias(c_symbol),
    pl.lit(resolution).alias(c_resolution),
)
with pl.Config(tbl_formatting="MARKDOWN"):
    print(
        df_ohlc.group_by(pl.col(c_symbol, c_resolution)).agg(
            pl.col(c_datetime_local).min().alias("min"),
            pl.col(c_datetime_local).max().alias("max"),
            pl.col(c_datetime_local).count().alias("count"),
        )
    )


shape: (1, 5)
| symbol | resolution | min                            | max                            | count |
| ---    | ---        | ---                            | ---                            | ---   |
| str    | str        | datetime[ms, America/New_York] | datetime[ms, America/New_York] | u32   |
|--------|------------|--------------------------------|--------------------------------|-------|
| ^GSPC  | 1wk        | 1927-12-26 00:00:00 EST        | 2026-02-16 00:00:00 EST        | 5121  |


In [37]:
staging_data = duckdb.sql(f"""
SELECT
{c_datetime_local},
{c_close},
(YEAR({c_datetime_local}) - 1729) % 27 high_cycle,
(YEAR({c_datetime_local}) - 1735) % 27 low_cycle,
(YEAR({c_datetime_local}) - 1911) % 54 panic_cycle,
LEAD({c_close}, 1, 0) OVER (ORDER BY {c_datetime_local}) - {c_close} AS forward_returns,
(LEAD({c_close}, 1, 0) OVER (ORDER BY {c_datetime_local}) - {c_close}) / {c_close} AS forward_roc,
FROM df_ohlc
""")

In [38]:
high_prices_highlights = alt.Chart(
    pl.DataFrame({
        'cycle_period': [0, 8, 8+9, 8+9+10],
    })
).mark_rule(color='red', strokeWidth=4, opacity=0.5).encode(
    x='cycle_period'
)

chart = duckdb.sql(f"""
SELECT
    high_cycle,
    mean(forward_returns) mean_returns,
    mean(forward_roc) mean_change,
from staging_data
group by high_cycle
order by high_cycle
""").pl().plot.bar(x="high_cycle", y="mean_change")

(chart + high_prices_highlights).save(EXPORT_PATH + "/high-prices.svg")
chart + high_prices_highlights


alt.LayerChart(...)

In [39]:
chart = duckdb.sql(f"""
select
year({c_datetime_local}) calendar_year,
mean(forward_roc) mean_weekly_roc,
case
    when high_cycle in [0, 8, 17] then 'high'
    when high_cycle in [1, 9, 18] then 'post_high'
end category
from staging_data
where (high_cycle in [0, 8, 17] or high_cycle in [1, 9, 18])
and year({c_datetime_local}) < year(now())
group by year({c_datetime_local}), category
order by year({c_datetime_local})
""").pl().plot.bar(x="calendar_year:N", y="mean_weekly_roc", color="category")
chart.save(EXPORT_PATH + "/post-high.svg")
chart

alt.Chart(...)